In [58]:
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split


movie_data_path = Path("movie_data.csv")
movie_data_df = pd.read_csv(movie_data_path)
movie_data_df.head()



/var/folders/v4/v1wh2j7n7_v1r1l0xj84m9xw0000gp/T/ipykernel_93301/1351858447.py:8: DtypeWarning: Columns (0,1,7,8,15,105,112,113,114,115,116,117,118,119,120,150,153,154,155,156,157,158,167,168,169,170,173,174,175,177,178,179,181,182,183,185,186,187,188,189) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_data_df = pd.read_csv(movie_data_path)


,Unnamed: 0,adult,backdrop_path,belongs_to_collection_id,belongs_to_collection_name,belongs_to_collection_poster_path,belongs_to_collection_backdrop_path,budget,genres_0_id,genres_0_name,...,production_companies_19_id,production_companies_19_logo_path,production_companies_19_name,production_companies_19_origin_country,production_companies_20_id,production_companies_20_logo_path,production_companies_20_name,production_companies_20_origin_country,origin_country_6,origin_country_7
0,0,False,/417tYZ4XUyJrtyZXj7HpvWf1E8f.jpg,1370345.0,The Wild Robot Collection,/cioNnsPSHJH9gsUSETPFHh0m6MT.jpg,/f6G8QPeod5ngQMs5Fe1O4LdphB7.jpg,78000000,16.0,Animation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,False,/3V4kLQg0kSqPLctI5ziYWabAZYF.jpg,558216.0,Venom Collection,/hoTLlTIohrzQ13HQVkZrDlvffuT.jpg,/vq340s8DxA5Q209FT8PHA6CXYOx.jpg,120000000,28.0,Action,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,False,/9SSEUrSqhljBMzRe4aBTh17rUaC.jpg,NaN,NaN,NaN,NaN,80000000,878.0,Science Fiction,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,False,/7h6TqPB3ESmjuVbxCxAeB1c9OB1.jpg,NaN,NaN,NaN,NaN,17500000,27.0,Horror,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,False,/xlkclSE4aq7r3JsFIJRgs21zUew.jpg,727761.0,Terrifier Collection,/4xIzrMcEvCzJm5qAl92WMHLSIeM.jpg,/zREjCmCHIHdEF6ufPoDQjhl4Wdm.jpg,2000000,27.0,Horror,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
# Load the movie credits data
movie_credit_path = Path("movie_credits_data.csv")
movie_credits_df = pd.read_csv(movie_credit_path)

movie_credits_df = movie_credits_df[movie_credits_df['known_for_department'] == 'Acting']

# Group by movie_id and get the top 3 popular actors for each movie
top_actors = movie_credits_df.sort_values(by='popularity', ascending=False).groupby('movie_id').head(3)

# Create a new dataframe with movie_id and top 3 actors
top_actors_df = top_actors.groupby('movie_id')['name'].apply(lambda x: x.tolist()).reset_index()

# Ensure each movie has exactly 3 actors by filling with None if less than 3
top_actors_df['top_3_actors'] = top_actors_df['name'].apply(lambda x: (x + [None] * 3)[:3])

# Split the list of actors into separate columns
top_actors_df[['actor_1', 'actor_2', 'actor_3']] = pd.DataFrame(top_actors_df['top_3_actors'].tolist(), index=top_actors_df.index)

# Drop the original list column
top_actors_df = top_actors_df.drop(columns=['name', 'top_3_actors'])

# Merge the top_actors_df with the original movie_credits_df
movie_credits_df = movie_credits_df.merge(top_actors_df, on='movie_id', how='left')

# Remove duplicate movie ids, keeping the first occurrence
movie_credits_df['movie_id'] = movie_credits_df['movie_id'].astype(int)
movie_credits_df = movie_credits_df.drop_duplicates(subset='movie_id', keep='first')

movie_credits_df = movie_credits_df[['movie_id', 'actor_1', 'actor_2', 'actor_3']].reset_index(drop=True)

# Display the first few rows
movie_credits_df.head()

,movie_id,actor_1,actor_2,actor_3
0,1184918,Pedro Pascal,Kit Connor,Catherine O'Hara
1,912649,Tom Hardy,Juno Temple,Andy Serkis
2,945961,Cailee Spaeny,Isabela Merced,Aileen Wu
3,933260,Margaret Qualley,Demi Moore,Dennis Quaid
4,1034541,David Howard Thornton,Lauren LaVera,Alexa Blair Robertson


In [94]:
# movie_data_df = movie_data_df[['budget', 'revenue', 'runtime', 'popularity', 'vote_average', 'vote_count', 'release_date', 'title', 'id', 'genres_0_name', 'genres_1_name', 'genres_2_name', 'production_companies_0_name', 'production_companies_1_name', 'production_companies_2_name', 'original_language', 'production_countries_0_name']]

movie_data_df = movie_data_df[['id', 'title', 'original_language', 'release_date', 'budget', 'revenue', 'runtime', 'popularity', 'vote_average', 'vote_count', 'genres_0_name', 'genres_1_name', 'genres_2_name', 'production_companies_0_name', 'production_companies_1_name', 'production_companies_2_name', 'production_countries_0_name']]

# Merge the movie_data_df with the movie_credits_df
movie_data_df = movie_data_df.merge(movie_credits_df, left_on='id', right_on='movie_id', how='left')

movie_data_df = movie_data_df.sort_values(by=['release_date'], ascending=False)
movie_data_df.head()

movie_data_df = movie_data_df.dropna()


In [95]:
movie_data_df['profit'] = pd.to_numeric(movie_data_df['revenue']) - pd.to_numeric(movie_data_df['budget'])
movie_data_df = movie_data_df[pd.to_numeric(movie_data_df['budget']) > 0]
movie_data_df['roi'] = pd.to_numeric(movie_data_df['profit']) / pd.to_numeric(movie_data_df['budget'])
movie_data_df.sort_values(by=['profit', 'vote_average', 'popularity'], ascending=False, inplace=True)

vote_average_theshold = movie_data_df['vote_average'].quantile(0.75)
popularity_theshold = movie_data_df['popularity'].quantile(0.75)

movie_data_df.head()


,id,title,original_language,release_date,budget,revenue,runtime,popularity,vote_average,vote_count,...,production_companies_0_name,production_companies_1_name,production_companies_2_name,production_countries_0_name,movie_id,actor_1,actor_2,actor_3,profit,roi
0,19995,Avatar,en,2009-12-15,237000000,2.923706e+09,162.0,129.244,7.583,31394.0,...,Dune Entertainment,Lightstorm Entertainment,20th Century Fox,United States of America,19995.0,Zoe Saldaña,Michelle Rodriguez,Sam Worthington,2.686706e+09,11.336312
1,634649,Spider-Man: No Way Home,en,2021-12-15,200000000,1.921847e+09,148.0,244.068,8.000,20001.0,...,Marvel Studios,Pascal Pictures,Columbia Pictures,United States of America,634649.0,Tom Hardy,Tom Holland,Andrew Garfield,1.721847e+09,8.609236
2,168259,Furious 7,en,2015-04-01,190000000,1.515400e+09,137.0,80.063,7.200,10543.0,...,Original Film,One Race,Universal Pictures,United States of America,168259.0,Jason Statham,Dwayne Johnson,Vin Diesel,1.325400e+09,6.975789
3,502356,The Super Mario Bros. Movie,en,2023-04-05,100000000,1.362000e+09,93.0,333.862,7.651,9037.0,...,Universal Pictures,Illumination,Nintendo,Japan,502356.0,Anya Taylor-Joy,Chris Pratt,Jack Black,1.262000e+09,12.620000
4,533535,Deadpool & Wolverine,en,2024-07-24,200000000,1.336816e+09,128.0,1900.830,7.700,4822.0,...,Marvel Studios,Maximum Effort,21 Laps Entertainment,United States of America,533535.0,Ryan Reynolds,Henry Cavill,Morena Baccarin,1.136816e+09,5.684081


In [96]:
movie_data_train, movie_data_test = train_test_split(movie_data_df, test_size=0.2, random_state=42)
movie_data_train.to_csv("movie_data_train.csv", index=False)
movie_data_test.to_csv("movie_data_test.csv", index=False)